In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future 

In [119]:
@macroexpand @model function smoothing(n_observations, noise_mean, noise_var)
    
    x_prior_mean = datavar(Float64)
    x_prior_var  = datavar(Float64)

    x_prior ~ NormalMeanVariance(x_prior_mean, x_prior_var) where { q = q(μ, v)q(x_prior) }

    x = randomvar(n_observations)
    y = datavar(Float64, n_observations)

    x[1] = x_prior

    for i in 2:n_observations
       x[i] ~ x[i - 1] + 1.0
       y[i] ~ x[i] + NormalMeanVariance(noise_mean, noise_var)
    end

    return x_prior_mean, x_prior_var, x, y
end

LoadError: LoadError: MethodError: no method matching interface_get_name(::Type{Val{:NormalMeanVariance}}, ::Type{Val{1}})
Closest candidates are:
  interface_get_name(::Type{Val{:NormalMeanVariance}}, !Matched::Type{Val{:out}}) at /Users/bvdmitri/.julia/dev/ReactiveMP/src/node.jl:365
  interface_get_name(::Type{Val{:NormalMeanVariance}}, !Matched::Type{Val{:μ}}) at /Users/bvdmitri/.julia/dev/ReactiveMP/src/node.jl:365
  interface_get_name(::Type{Val{:NormalMeanVariance}}, !Matched::Type{Val{:mean}}) at /Users/bvdmitri/.julia/dev/ReactiveMP/src/node.jl:367
  ...
in expression starting at In[119]:1

In [91]:
?allunique

search: allunique



```
allunique(itr) -> Bool
```

Return `true` if all values from `itr` are distinct when compared with [`isequal`](@ref).

# Examples

```jldoctest
julia> a = [1; 2; 3]
3-element Array{Int64,1}:
 1
 2
 3

julia> allunique([a, a])
false
```


In [86]:
noise_real_mean = -10.0
noise_real_var  = 10.0

n = 500
data = map(d -> d + rand(Normal(noise_real_mean, sqrt(noise_real_var))), collect(1:n));

In [87]:
function inference(; data, x_prior)
    n = length(data)
    
    x_prior_mean, x_prior_var, x, y = smoothing(n, noise_real_mean, noise_real_var);

    subscriptions = Vector{Teardown}(undef, n)
    marginals = Vector{Marginal}(undef, n)
    
    for i in 1:n
         @inbounds subscriptions[i] = subscribe!(getmarginal(x[i]), (m) -> marginals[i] = m)
    end
    
    update!(x_prior_mean, mean(x_prior))
    update!(x_prior_var, var(x_prior))
    
    for i in 1:n
        @inbounds update!(y[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return marginals
end

inference (generic function with 1 method)

In [80]:
inference(
    data = data,
    x_prior = NormalMeanVariance(0.0, 10000.0)
)

LoadError: UndefVarError: q not defined

In [32]:
using MacroTools

In [33]:
MacroTools.@capture(:(q = q(1)q(2)q(3)), q = *(f__))

true

In [106]:
dump(:(x[i]))

┌ Error: Failed to revise /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl
│   exception = (LoadError("/Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl", 162, "incomplete: \"module\" at /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl:1 requires end"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000013cc25e8e])
└ @ Revise /Users/bvdmitri/.julia/packages/Revise/nWJXk/src/packagedef.jl:693
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /Users/bvdmitri/.julia/dev/GraphPPL.jl/src/GraphPPL.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /Users/bvdmitri/.julia/packages/Revise/nWJXk/src/packagedef.jl:789


Expr
  head: Symbol ref
  args: Array{Any}((2,))
    1: Symbol x
    2: Symbol i


In [117]:
findfirst(==(:a), [ :a, :b, :c ])

1

In [114]:
?findnext

search: findnext find_index



```
findnext(A, i)
```

Find the next index after or including `i` of a `true` element of `A`, or `nothing` if not found.

Indices are of the same type as those returned by [`keys(A)`](@ref) and [`pairs(A)`](@ref).

# Examples

```jldoctest
julia> A = [false, false, true, false]
4-element Array{Bool,1}:
 0
 0
 1
 0

julia> findnext(A, 1)
3

julia> findnext(A, 4) # returns nothing, but not printed in the REPL

julia> A = [false false; true false]
2×2 Array{Bool,2}:
 0  0
 1  0

julia> findnext(A, CartesianIndex(1, 1))
CartesianIndex(2, 1)
```

---

```
findnext(predicate::Function, A, i)
```

Find the next index after or including `i` of an element of `A` for which `predicate` returns `true`, or `nothing` if not found.

Indices are of the same type as those returned by [`keys(A)`](@ref) and [`pairs(A)`](@ref).

# Examples

```jldoctest
julia> A = [1, 4, 2, 2];

julia> findnext(isodd, A, 1)
1

julia> findnext(isodd, A, 2) # returns nothing, but not printed in the REPL

julia> A = [1 4; 2 2];

julia> findnext(isodd, A, CartesianIndex(1, 1))
CartesianIndex(1, 1)
```

---

```
findnext(pattern::AbstractString, string::AbstractString, start::Integer)
findnext(pattern::Regex, string::String, start::Integer)
```

Find the next occurrence of `pattern` in `string` starting at position `start`. `pattern` can be either a string, or a regular expression, in which case `string` must be of type `String`.

The return value is a range of indices where the matching sequence is found, such that `s[findnext(x, s, i)] == x`:

`findnext("substring", string, i)` == `start:stop` such that `string[start:stop] == "substring"` and `i <= start`, or `nothing` if unmatched.

# Examples

```jldoctest
julia> findnext("z", "Hello to the world", 1) === nothing
true

julia> findnext("o", "Hello to the world", 6)
8:8

julia> findnext("Lang", "JuliaLang", 2)
6:9
```

---

```
findnext(ch::AbstractChar, string::AbstractString, start::Integer)
```

Find the next occurrence of character `ch` in `string` starting at position `start`.

!!! compat "Julia 1.3"
    This method requires at least Julia 1.3.


# Examples

```jldoctest
julia> findnext('z', "Hello to the world", 1) === nothing
true

julia> findnext('o', "Hello to the world", 6)
8
```
